In [ ]:
from mlx_lm import load, generate
# model, tokenizer = load("mlx-community/NeuralBeagle14-7B-4bit-mlx")

model, tokenizer = load("mlx-community/NeuralBeagle14-7B-mlx")

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a helpfull assistent.",
    },
    {"role": "user", "content": "tell me a joke about a neural network"},
]
chat_prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True)

print(chat_prompt)

In [ ]:
response = generate(model, tokenizer, prompt=chat_prompt,
                    verbose=False, max_tokens=400)

In [ ]:
# find the next <|im_end|> in the response and remove everything after it
response = response.split("<|im_end|>")[0]
print(response)

# Using Functions for chatting with the bot

In [ ]:
from mlx_lm import load, generate
# model, tokenizer = load("mlx-community/NeuralBeagle14-7B-4bit-mlx")

model, tokenizer = load("mlx-community/NeuralBeagle14-7B-mlx")

In [ ]:
# define a function which takes the response and generates a condensed version of it as context for the next messages
context_model, context_tokenizer = load(
    "mlx-community/NeuralBeagle14-7B-4bit-mlx")


def generate_context(response, max_tokens=400):
    messages = [
        {
            "role": "system",
            "content": "You are a helpfull assistent. Summarizing messages.",
        },
        {"role": "message", "content": response},
    ]

    chat_prompt = context_tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True)

    response = generate(context_model, context_tokenizer, prompt=chat_prompt,
                        verbose=False, max_tokens=max_tokens)
    
    # find the next <|im_end|> in the response and remove everything after it
    response = response.split("<|im_end|>")[0]
    return response

In [ ]:
def chat(prompt, context_vector=None, max_tokens=1000):
    # Combine the context vector with the prompt if the context is not empty
    if context_vector is not None and len(context_vector) > 0:
        combined_prompt = [
            {"role": "system", "content": "You are a helpfull assistent."},
            {"role": "context", "content":  context_vector}, 
            {"role": "user", "content": prompt}]
    else:
        combined_prompt = [
            {"role": "system", "content": "You are a helpfull assistent."},
            {"role": "user", "content": prompt}]

    combined_prompt = tokenizer.apply_chat_template(
        combined_prompt, tokenize=False, add_generation_prompt=True)

    # Generate the response using the model
    response = generate(model, tokenizer, prompt=combined_prompt,
                        verbose=False, max_tokens=max_tokens)
    response = response.split("<|im_end|>")[0]

    return response

In [ ]:
response = chat("tell me a joke about a neural network", max_tokens=400)
print(response)
new_context_vector = generate_context(response, max_tokens=100)

In [ ]:
new_context_vector

In [ ]:
response = chat("tell me another joke about a neural network", new_context_vector)
print(response)

# Test Class for the chatbot

In [ ]:
from chatter_mlx import Chatter

In [ ]:
# chatbot = Chatter("mlx-community/CodeLlama-7b-Python-4bit-MLX")
chatbot = Chatter("mlx-community/NeuralBeagle14-7B-4bit-mlx")

In [ ]:
response, context = chatbot.chat(
    "Write me a function which that takes an array of numbers and sorts them in ascending order using the bubble sort algorithm in plain python code.", max_tokens=800)
print(response)

In [ ]:
context

In [ ]:
response, context = chatbot.chat("Now write this function in C++.", context, max_tokens=1000)

In [ ]:
print(response)

In [ ]:
context

In [ ]:
chatbot.chat("Whats the difference between the algorithms you provided?", context, max_tokens=200)